In [ ]:
#@title # 설치하기
#@markdown 약 1분 40초 ~ 2분 정도 걸립니다.

# if error occurs, remove > /dev/null 2>&1 and test
print("PYTHON...")
!apt-get install -y python3.10  > /dev/null 2>&1
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1 > /dev/null 2>&1
!update-alternatives --set python3 /usr/bin/python3.10 > /dev/null 2>&1
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3 > /dev/null 2>&1
!pip install pip==23.3.1 > /dev/null 2>&1

!apt-get install -y ffmpeg > /dev/null 2>&1
print('FFMPEG...')

!pip install uv > /dev/null 2>&1
print('UV...')

!git clone https://github.com/tsyqax/RVC-Cover-Maker-NoGUI.git DIR > /dev/null 2>&1
%cd DIR
print('GITHUB...')

print('INSTALLING...')
#!uv pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118  --system
!uv pip install -r requirements.txt --system > /dev/null 2>&1

!uv pip install torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 --index-url https://download.pytorch.org/whl/cu118 --system > /dev/null 2>&1
!pip install demucs > /dev/null 2>&1
!python model_down.py

print('DONE!')

PYTHON...
FFMPEG...
UV...
/content/DIR
GITHUB...
INSTALLING...
MODEL... MAY 2 MINUTES
DONE!


In [ ]:
#@title #**모델 다운로드**
#@markdown 허깅페이스 또는 드라이브 링크
url = "https://huggingface.co/jarari/RVC-v2/resolve/main/seele_kr.zip"  # @param {type:"string"}
#@markdown 모델 이름
dir_name = "seele_kr"  # @param {type:"string"}

import os
import urllib.request
import zipfile
import shutil
import gdown

#filename = f'{dir_name}.zip'
models_dir = os.path.join(os.getcwd(), 'models') # CURRENT / models /
download_file = os.path.join(models_dir, f'{dir_name}.zip') # CURRENT / models / dirname.zip
download_dir = os.path.join(models_dir, dir_name) # CURRENT / models / dir_name /
output = os.path.join(os.getcwd(), 'output') # CURRENT / output

os.makedirs(models_dir, exist_ok=True)

print('STARTING...!')

def unzip2(filename, output):
  os.makedirs(output, exist_ok=True)
  with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall(output)
  os.remove(filename)

if 'huggingface.co' in url: # hugginface
  try:
    zipname = url.split('/')[-1]
    print("DOWNLOADING...!!")

    if os.path.exists(download_dir):
      print(f'RE---DOWNLOADING...!!')
      shutil.rmtree(download_dir)

    urllib.request.urlretrieve(url, download_file)

    print('UNZIPPING..!!!')
    unzip2(download_file, download_dir)

    print('DOWNLOADED!!')

  except Exception as e:
     raise Exception(str(e))

elif 'drive.google.com' in url:
  file_id = url.split('/')[-2]
  try:
    print("DOWNLOADING...!!")

    if os.path.exists(download_dir):
      print(f'RE---DOWNLOADING...!!')
      shutil.rmtree(download_dir)

    gdown.download(f'https://drive.google.com/uc?id={file_id}', download_file, quiet=False)
    unzip2(download_file, download_dir)
    print('DOWNLOADED!!')

  except Exception as e:
      raise Exception(str(e))
else:
  print('InValid LINK... HuggingFace or Google Drive is OKAY')

STARTING...!
DOWNLOADING...!!
UNZIPPING..!!!
DOWNLOADED!!


In [ ]:
#@title #**추론하기**

#@markdown 파일명(정상작동) 또는 유튜브 URL(아직 테스트 못함)<br>
#@markdown 드라이브 경로는 상대경로로 적어도 괜찮습니다
input = "/content/drive/MyDrive/RVC_Input/ego2.mp3"  # @param {type:"string"}

#@markdown RVC 이름
dir_name = "seele_kr"  # @param {type:"string"}

#@markdown 음원 분리 여부
sep_mode = True  # @param {type:"boolean"}

#@markdown 보컬 피치 조절 ( 삼겹살  = 반음(semiton) * 1.2, 10 삼겹살 = 1 옥타브)
pitch_vocal = 3  # @param {type:"slider", min:-15, max:15, step:0.5}

#@markdown 나머지 피치 조절 (*분리를 하지 않는다면, 영향을 주지 않습니다)
pitch_other = 1.5  # @param {type:"slider", min:-15, max:15, step:0.5}

#@markdown 보컬 음량 조절
vocal_sound = 100  # @param {type:"slider", min:0, max:150, step:5}

#@markdown 나머지 음량 조절
other_sound = 75  # @param {type:"slider", min:0, max:150, step:5}

# @markdown 추론 알고리즘
rvc_method = "rmvpe"  # @param ["rmvpe", "fcpe"] {allow-input: false}

#@markdown 인덱스 비율 (기본 0.75)
rvc_index_rate = 0.75  # @param {type:"slider", min:0, max:1, step:0.05}

#@markdown 음량 안정화 (기본 0.3)
rvc_rmx_rate = 0.3  # @param {type:"slider", min:0, max:1, step:0.1}

# Pass the defined variables as command-line arguments to main.py

if 'https://' not in input and 'http://' not in input:
  if '/content/' not in input:
    input = f'/content/drive/MyDrive/{input}'
  from google.colab import drive
  drive.mount('/content/drive')

'''
import subprocess

command = [
    "python",
    "main.py",
    "--input",
    f"{input}",
    "--rvc-name",
    f"{dir_name}",
    "--sep-mode",
    f"{sep_mode}",
    "--pitch-vocal",
    f"{pitch_vocal}",
    "--pitch-other",
    f"{pitch_other}",
    "--vocal-sound",
    f"{vocal_sound}",
    "--other-sound",
    f"{other_sound}",
    "--rvc-method",
    f"{rvc_method}",
    "--index-rate",
    f"{rvc_index_rate}",
    "--rmx-rate",
    f"{rvc_rmx_rate}",
]
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)

# Print the output in real-time
for line in process.stdout:
    print(line, end='')

# Wait for the process to finish
process.wait()'''

!python main.py \
    --input "{input}" \
    --rvc-name "{dir_name}" \
    --sep-mode {sep_mode} \
    --pitch-vocal {pitch_vocal} \
    --pitch-other {pitch_other} \
    --vocal-sound {vocal_sound} \
    --other-sound {other_sound} \
    --rvc-method "{rvc_method}" \
    --index-rate {rvc_index_rate} \
    --rms-rate {rvc_rmx_rate}